In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Generalized parameter-shift rules {#general_parshift}
=================================

::: {.meta}
:property=\"og:description\": Reconstruct quantum functions and compute
their derivatives. :property=\"og:image\":
<https://pennylane.ai/qml/_images/thumbnail_genpar.png>
:::

::: {.related}
tutorial\_expressivity\_fourier\_series Quantum models as Fourier series
tutorial\_rotoselect Quantum circuit structure learning
tutorial\_quantum\_analytic\_descent Quantum analytic descent
:::

*Author: David Wierichs (Xanadu Resident) --- Posted: 23 August 2021.
Last updated: 27 August 2021*

In this demo we will look at univariate quantum functions, i.e., those
that depend on a single parameter. We will investigate the form such
functions usually take and demonstrate how we can *reconstruct* them as
classical functions, capturing the full dependence on the input
parameter. Once we have this reconstruction, we use it to compute
analytically exact derivatives of the quantum function. We implement
this in two ways: first, by using autodifferentiation on the classical
function that is produced by the reconstruction, which is flexible with
respect to the degree of the derivative. Second, by computing the
derivative manually, resulting in generalized parameter-shift rules for
quantum functions that is more efficient (regarding classical cost) than
the autodifferentiation approach, but requires manual computations if we
want to access higher-order derivatives. All we will need for the demo
is the insight that these functions are Fourier series in their
variable, and the reconstruction itself is a [trigonometric
interpolation](https://en.wikipedia.org/wiki/Trigonometric_interpolation).

A full description of the reconstruction, the technical derivation of
the parameter-shift rules, and considerations for multivariate functions
can be found in the paper [General parameter-shift rules for quantum
gradients](https://arxiv.org/abs/2107.12390) . The core idea to consider
these quantum functions as Fourier series was first presented in the
preprint [Calculus on parameterized quantum
circuits](https://arxiv.org/abs/1812.06323). We will follow, but there
also are two preprints discussing general parameter-shift rules: an
algebraic approach in [Analytic gradients in variational quantum
algorithms: Algebraic extensions of the parameter-shift rule to general
unitary transformations](https://arxiv.org/abs/2107.08131) and one
focusing on special gates and spectral decompositions, namely
[Generalized quantum circuit differentiation
rules](https://arxiv.org/abs/2108.01218) .

| 

![Function reconstruction and differentiation via parameter
shifts.](../demonstrations/general_parshift/thumbnail_genpar.png){.align-center
width="50.0%"}

Cost functions arising from quantum gates
-----------------------------------------

We start our investigation by considering a cost function that arises
from measuring the expectation value of an observable in a quantum
state, created with a parametrized quantum operation that depends on a
single variational parameter $x$. That is, the state may be prepared by
any circuit, but we will only allow a single parameter in a single
operation to enter the circuit. For this we will use a handy gate
structure that allows us to tune the complexity of the operation --- and
thus of the cost function. More concretely, we initialize a qubit
register in a random state $|\psi\rangle$ and apply a layer of Pauli-$Z$
rotations `RZ` to all qubits, where all rotations are parametrized by
the *same* angle $x$. We then measure the expectation value of a random
Hermitian observable $B$ in the created state, so that our cost function
overall has the form

Here, $U(x)$ consists of a layer of `RZ` gates,

Let\'s implement such a cost function using PennyLane. We begin with
functions that generate the random initial state $|\psi\rangle$ and the
random observable $B$ for a given number of qubits $N$ and a fixed seed:


Now let\'s set up a \"cost function generator\", namely a function that
will create the `cost` function we discussed above, using $|\psi\rangle$
as initial state and measuring the expectation value of $B$. This
generator has the advantage that we can quickly create the cost function
for various numbers of qubits --- and therefore cost functions with
different complexity.

We will use the default qubit simulator with its JAX backend and also
will rely on the NumPy implementation of JAX. To obtain precise results,
we enable 64-bit `float` precision via the JAX config.


We also prepare some plotting functionalities and colors:


In [ ]:
# Set a plotting range on the x-axis


# Colors

Now that we took care of these preparations, let\'s dive right into it:
It can be shown that $E(x)$ takes the form of a Fourier series in the
variable $x$. That is to say that

Here, $a_{\ell}$ and $b_{\ell}$ are the *Fourier coefficients*. If you
would like to understand this a bit better still, have a read of
`~.pennylane.fourier`{.interpreted-text role="mod"} and remember to
check out the
`Fourier module tutorial </demos/tutorial_expressivity_fourier_series>`{.interpreted-text
role="doc"}.

Due to $B$ being Hermitian, $E(x)$ is a real-valued function, so only
positive frequencies and real coefficients appear in the Fourier series
for $E(x)$. This is true for any number of qubits (and therefore `RZ`
gates) we use.

Using our function `make_cost` from above, we create the cost function
for several numbers of qubits and store both the function and its
evaluations on the plotting range `X`.


In [ ]:
# Qubit numbers

# Fix a seed

Let\'s take a look at the created $E(x)$ for the various numbers of
qubits:


In [ ]:
# Figure with multiple axes

| 

Indeed we see that $E(x)$ is a periodic function whose complexity grows
when increasing $N$ together with the number of `RZ` gates. To take a
look at the frequencies that are present in these functions, we may use
PennyLane\'s `~.pennylane.fourier`{.interpreted-text role="mod"} module.


The number of positive frequencies that appear in $E(x)$ is the same as
the number of `RZ` gates we used in the circuit! Recall that we only
need to consider the positive frequencies because $E(x)$ is real-valued,
and that we accounted for the zero-frequency contribution in the
coefficient $a_0$. If you are interested why the number of gates
coincides with the number of frequencies, check out the
`Fourier module tutorial </demos/tutorial_expressivity_fourier_series>`{.interpreted-text
role="doc"}.

Before moving on, let\'s also have a look at the Fourier coefficients in
the functions we created:


We find the real (imaginary) Fourier coefficients to be
(anti-)symmetric. This is expected because $E(x)$ is real-valued and we
again see why it is enough to consider positive frequencies: the
coefficients of the negative frequencies follow from those of the
positive frequencies.

Determining the full dependence on $x$
======================================

Next we will show how to determine the *full* dependence of the cost
function on $x$, i.e., we will *reconstruct* $E(x)$. The key idea is not
new: Since $E(x)$ is periodic with known, integer frequencies, we can
reconstruct it *exactly* by using trigonometric interpolation. For this,
we evaluate $E$ at shifted positions $x_\mu$. We will show the
reconstruction both for *equidistant* and random shifts, corresponding
to a [uniform](https://en.wikipedia.org/wiki/Discrete_Fourier_transform)
and a
[non-uniform](https://en.wikipedia.org/wiki/Non-uniform_discrete_Fourier_transform)
discrete Fourier transform (DFT), respectively.

Equidistant shifts
------------------

For the equidistant case we can directly implement the trigonometric
interpolation:

where we reformulated $E$ in the second expression using the [sinc
function](https://en.wikipedia.org/wiki/Sinc_function) to enhance the
numerical stability. Note that we have to take care of a rescaling
factor of $\pi$ between this definition of $\operatorname{sinc}$ and the
NumPy implementation `np.sinc`.


So how is this reconstruction doing? We will plot it along with the
original function $E$, mark the shifted evaluation points $x_\mu$ (with
crosses), and also show its deviation from $E(x)$ (lower plots). For
this, a function for the whole procedure of comparing the functions
comes in handy, and we will reuse it further below. For convenience,
showing the deviation will be an optional feature controled by the
`show_diff` keyword argument.


*It works!*

Non-equidistant shifts
======================

Now let\'s test the reconstruction with less regular sampling points on
which to evaluate $E$. This means we can no longer use the closed-form
expression from above, but switch to solving the set of equations

with the---now irregular---sampling points $x_\mu$. For this, we set up
the matrix

collect the Fourier coefficients of $E$ into the vector
$\boldsymbol{W}=(a_0, \boldsymbol{a}, \boldsymbol{b})$, and the
evaluations of $E$ into another vector called $\boldsymbol{E}$ so that

Let\'s implement this right away! We will take the function and the
shifts $x_\mu$ as inputs, inferring $R$ from the number of the provided
shifts, which is $2R+1$.


To see this version of the reconstruction in action, we will sample the
shifts $x_\mu$ at random in $[-\pi,\pi)$:


Again, we obtain a perfect reconstruction of $E(x)$ up to numerical
errors. We see that the deviation from the original cost function became
larger than for equidistant shifts for some of the qubit numbers but it
still remains much smaller than any energy scale of relevance in
applications. The reason for these larger deviations is that some
evaluation positions $x_\mu$ were sampled very close to each other, so
that inverting the matrix $C$ becomes less stable numerically.
Conceptually, we see that the reconstruction does *not* rely on
equidistant evaluations points.

Differentiation via reconstructions
===================================

Next, we look at a modified reconstruction strategy that only obtains
the odd or even part of $E(x)$. This can be done by slightly modifying
the shifted positions at which we evaluate $E$ and the kernel functions.

From a perspective of implementing the derivatives there are two
approaches, differing in which parts we derive on paper and which we
leave to the computer: In the first approach, we perform a partial
reconstruction using the evaluations of the original cost function $E$
on the quantum computer, as detailed below. This gives us a function
implemented in `jax.numpy` and we may afterwards apply `jax.grad` to
this function and obtain the derivative function. $E(0)$ then is only
one evaluation of this function away. In the second approach, we compute
the derivative of the partial reconstructions *manually* and directly
implement the resulting shift rule that multiplies the quantum computer
evaluations with coefficients and sums them up. This means that the
partial reconstruction is not performed at all by the classical
computer, but only was used on paper to derive the formula for the
derivative.

*Why do we look at both approaches?*, you might ask. That is because
neither of them is better than the other for *all* applications. The
first approach offers us derivatives of any order without additional
manual work by iteratively applying `jax.grad`, which is very
convenient. However, the automatic differentiation via JAX becomes
increasingly expensive with the order and we always reconstruct the
*same* type of function, namely Fourier series, so that computing the
respective derivatives once manually and coding up the resulting
coefficients of the parameter-shift rule pays off in the long run. This
is the strength of the second approach. We start with the first
approach.

Automatically differentiated reconstructions
--------------------------------------------

We implement the partial reconstruction method as a function; using
PennyLane\'s automatic differentiation backends, this then enables us to
obtain the derivatives at the point of interest. For odd-order
derivatives, we use the reconstruction of the odd part, for the
even-order derivatives that of the even part.

We make use of modified [Dirichlet
kernels](https://en.wikipedia.org/wiki/Dirichlet_kernel)
$\tilde{D}_\mu(x)$ and equidistant shifts for this. For the odd
reconstruction we have

which we can implement using the reformulation

for the kernel.


In [ ]:
# Odd linear combination of Dirichlet kernels

The even part on the other hand takes the form

Note that not only the kernels $\hat{D}_\mu(x)$ but also the shifted
positions $\{x_\mu\}$ differ between the odd and even case.


In [ ]:
# Even linear combination of Dirichlet kernels









# Special cases of even kernels

We also set up a function that performs both partial reconstructions and
sums the resulting functions to the full Fourier series.


We show these even (blue) and odd (red) reconstructions and how they
indeed sum to the full function (orange, dashed). We will again use the
`compare_functions` utility from above for the comparison.


In [ ]:
# Obtain the shifts for the reconstruction of both parts











# Show the reconstructed parts and the sums

Great! The even and odd part indeed sum to the correct function again.
But what did we gain?

Nothing, actually, for the full reconstruction! Quite the opposite, we
spent $2R$ evaluations of $E$ on each part, that is $4R$ evaluations
overall to obtain a description of the full function $E$! This is way
more than the $2R+1$ evaluations needed for the full reconstructions
from the beginning.

However, remember that we set out to compute derivatives of $E$ at $0$,
so that for derivatives of odd/even order only the odd/even
reconstruction is required. Using an autodifferentiation framework, e.g.
JAX, we can easily compute such higher-order derivatives:


In [ ]:
# An iterative function computing the ``order``th derivative of a function ``f`` with JAX


# Compute the first, second, and fourth derivative

The derivatives coincide.

Generalized parameter-shift rules
=================================

The second method is based on the previous one. Instead of consulting
JAX, we may compute the wanted derivative of the odd/even kernel
function manually and thus derive general parameter-shift rules from
this. We will leave the technical derivation of these rules to the
paper. Start with the first derivative, which certainly is used the
most:

This is straight-forward to implement by defining the coefficients and
evaluating $E$ at the shifted positions $x_\mu$:


The second-order derivative takes a similar form, but we have to take
care of the evaluation at $0$ and the corresponding coefficient
separately:

Let\'s code this up, again we only get slight complications from the
special evaluation at $0$:


We will compare these two shift rules to the finite-difference
derivative commonly used for numerical differentiation. We choose a
finite difference of $d_x=5\times 10^{-5}$.


All that is left is to compare the computed parameter-shift and
finite-difference derivatives:


The parameter-shift rules work as expected! And we were able to save a
circuit evaluation as compared to a full reconstruction.

And this is all we want to show here about univariate function
reconstructions and generalized parameter shift rules. Note that the
techniques above can partially be extended to frequencies that are not
integer-valued, but many closed form expressions are no longer valid.
For the reconstruction, the approach via Dirichlet kernels no longer
works in the general case; instead, a system of equations has to be
solved, but with generalized frequencies $\{\Omega_\ell\}$ instead of
$\{\ell\}$ (see e.g. Sections III A-C in[^1])

References
==========

[^1]: David Wierichs, Josh Izaac, Cody Wang, Cedric Yen-Yu Lin.
    \"General parameter-shift rules for quantum gradients\". [arXiv
    preprint arXiv:2107.12390](https://arxiv.org/abs/2107.12390).


About the author
================
